In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1. Create Dataset

In [2]:
# Generate synthetic dataset (binary classification)
X, y = make_classification(
    n_samples=1000, n_features=2, n_classes=2,
    n_informative=2, n_redundant=0, random_state=42
)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale features (best practice for ANNs)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# 2. Custom Dataset Class

In [3]:
class ClassificationDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.long)  # classification → long

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Create train/test datasets
train_dataset = ClassificationDataset(X_train, y_train)
test_dataset = ClassificationDataset(X_test, y_test)

# 3. DataLoader with Important Parameters

In [8]:
train_loader = DataLoader(
    train_dataset,
    batch_size=32,          # mini-batch training
    shuffle=True,           # shuffle for randomness
    num_workers=2,          # parallel data loading (if CPU cores available)
    pin_memory=True         # better performance when using GPU
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False           # no shuffle for evaluation
)

# 4. Define a Simple ANN

In [4]:
class SimpleANN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleANN, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# Instantiate model
model = SimpleANN(input_dim=2, hidden_dim=16, output_dim=2)

# 5. Loss & Optimizer

In [5]:
criterion = nn.CrossEntropyLoss()       # classification loss
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 6. Training Loop

In [9]:
def train(model, loader, criterion, optimizer, epochs=20):
    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for X_batch, y_batch in loader:
            # Forward
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # Metrics
            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        acc = 100 * correct / total
        print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss/len(loader):.4f}, Acc: {acc:.2f}%")

train(model, train_loader, criterion, optimizer, epochs=20)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch 1/20, Loss: 0.4984, Acc: 75.38%
Epoch 2/20, Loss: 0.3225, Acc: 86.12%
Epoch 3/20, Loss: 0.3148, Acc: 86.62%
Epoch 4/20, Loss: 0.3079, Acc: 87.25%
Epoch 5/20, Loss: 0.2972, Acc: 87.88%
Epoch 6/20, Loss: 0.2943, Acc: 88.00%
Epoch 7/20, Loss: 0.2876, Acc: 87.88%
Epoch 8/20, Loss: 0.2835, Acc: 88.88%
Epoch 9/20, Loss: 0.2810, Acc: 88.38%
Epoch 10/20, Loss: 0.2746, Acc: 88.50%
Epoch 11/20, Loss: 0.2701, Acc: 89.25%
Epoch 12/20, Loss: 0.2637, Acc: 89.50%
Epoch 13/20, Loss: 0.2590, Acc: 89.88%
Epoch 14/20, Loss: 0.2553, Acc: 90.00%
Epoch 15/20, Loss: 0.2486, Acc: 90.00%
Epoch 16/20, Loss: 0.2428, Acc: 90.50%
Epoch 17/20, Loss: 0.2396, Acc: 90.62%
Epoch 18/20, Loss: 0.2378, Acc: 90.88%
Epoch 19/20, Loss: 0.2321, Acc: 90.75%
Epoch 20/20, Loss: 0.2293, Acc: 91.12%


# 7. Evaluation

In [10]:
def evaluate(model, loader):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for X_batch, y_batch in loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    print(f"Test Accuracy: {100 * correct / total:.2f}%")

evaluate(model, test_loader)

Test Accuracy: 92.50%
